# LQR Control for TurtleBot3: Optimal Point Stabilization

This notebook demonstrates **Linear Quadratic Regulator (LQR)** control for point stabilization on the TurtleBot3 differential drive robot. We show how LQR provides optimal state feedback control and compare it with traditional PID control.

## Problem Setup

### Point Stabilization Challenge

Given a differential drive robot like the TurtleBot3, we want to:
- Drive the robot to a target position $(x_{goal}, y_{goal})$
- Align with a desired orientation $\theta_{goal}$
- Minimize control effort and tracking error
- Handle constraints smoothly (velocity limits)

LQR provides an optimal solution by:
- Minimizing a quadratic cost function balancing tracking error and control effort
- Computing optimal state feedback gains
- Guaranteeing stability for linear systems

## Mathematical Formulation

### Unicycle Dynamics

The TurtleBot3 follows unicycle (differential drive) dynamics:

$$
\begin{aligned}
\dot{x} &= v \cos(\theta) \\
\dot{y} &= v \sin(\theta) \\
\dot{\theta} &= \omega
\end{aligned}
$$

where:
- $(x, y, \theta)$: Position and orientation
- $v$: Linear velocity (control input)
- $\omega$: Angular velocity (control input)

### Linearization

To apply LQR (a linear control method), we linearize around the equilibrium (goal state):

Define error state:
$$
\mathbf{e} = \begin{bmatrix} x - x_{goal} \\ y - y_{goal} \\ \theta - \theta_{goal} \end{bmatrix}
$$

Linearized dynamics in discrete time:
$$
\mathbf{e}_{k+1} = A \mathbf{e}_k + B \mathbf{u}_k
$$

where:
$$
A = \begin{bmatrix}
1 & 0 & -v_{ref} \Delta t \sin(\theta_{ref}) \\
0 & 1 & v_{ref} \Delta t \cos(\theta_{ref}) \\
0 & 0 & 1
\end{bmatrix}, \quad
B = \begin{bmatrix}
\Delta t \cos(\theta_{ref}) & 0 \\
\Delta t \sin(\theta_{ref}) & 0 \\
0 & \Delta t
\end{bmatrix}
$$

For point stabilization ($v_{ref} = 0$), this simplifies.

### LQR Optimization

LQR solves:
$$
\min_{\mathbf{u}_0, \ldots, \mathbf{u}_\infty} \sum_{k=0}^\infty \left( \mathbf{e}_k^T Q \mathbf{e}_k + \mathbf{u}_k^T R \mathbf{u}_k \right)
$$

subject to: $\mathbf{e}_{k+1} = A \mathbf{e}_k + B \mathbf{u}_k$

The optimal control is:
$$
\mathbf{u}_k^* = -K \mathbf{e}_k
$$

where $K$ is computed from the Discrete Algebraic Riccati Equation (DARE):
$$
P = Q + A^T P A - A^T P B (R + B^T P B)^{-1} B^T P A
$$
$$
K = (R + B^T P B)^{-1} B^T P A
$$

## Implementation

### System Parameters

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve_discrete_are

# Simulation parameters
dt = 0.1  # 100 ms control loop
T_sim = 20.0  # 20 second simulation
N_steps = int(T_sim / dt)

# Robot parameters (TurtleBot3 Burger)
V_MAX = 0.22  # m/s (max linear velocity)
OMEGA_MAX = 2.84  # rad/s (max angular velocity)

# Goal states (multiple waypoints)
goals = [
    np.array([2.0, 0.0, 0.0]),      # Move forward
    np.array([2.0, 2.0, np.pi/2]),  # Move up
    np.array([0.0, 2.0, np.pi]),    # Move left
    np.array([0.0, 0.0, -np.pi/2])  # Return home
]

print(f"Simulation parameters:")
print(f"  Time step: {dt} s")
print(f"  Duration: {T_sim} s")
print(f"  Robot velocity limits: v_max={V_MAX} m/s, omega_max={OMEGA_MAX} rad/s")
print(f"  Waypoints: {len(goals)} goals")

### Linearized Dynamics

In [ ]:
def compute_linearized_dynamics(theta_ref, v_ref, dt):
    """
    Compute linearized unicycle dynamics around reference state.
    
    Args:
        theta_ref: Reference orientation (rad)
        v_ref: Reference linear velocity (m/s)
        dt: Time step (s)
    
    Returns:
        A, B: Linearized system matrices
    """
    # State: [x, y, theta]
    # Control: [v, omega]
    
    A = np.array([
        [1, 0, -v_ref * dt * np.sin(theta_ref)],
        [0, 1, v_ref * dt * np.cos(theta_ref)],
        [0, 0, 1]
    ])
    
    B = np.array([
        [dt * np.cos(theta_ref), 0],
        [dt * np.sin(theta_ref), 0],
        [0, dt]
    ])
    
    return A, B

# Example: Linearization at theta=0, v=0 (point stabilization)
A_0, B_0 = compute_linearized_dynamics(theta_ref=0.0, v_ref=0.1, dt=dt)

print("Linearized dynamics (theta=0, v=0.1):")
print(f"A =\n{A_0}")
print(f"B =\n{B_0}")

### LQR Gain Computation

In [ ]:
from pykal.algorithm_library.controllers.lqr import LQR
def compute_lqr_gain(A, B, Q, R):
    """
    Compute LQR gain using DARE.
    
    Args:
        A, B: System matrices
        Q: State cost matrix (n x n)
        R: Control cost matrix (m x m)
    
    Returns:
        K: LQR gain matrix (m x n)
        P: Solution to DARE (n x n)
    """
    # Solve DARE
    P = solve_discrete_are(A, B, Q, R)
    
    # Compute gain
    K = np.linalg.solve(R + B.T @ P @ B, B.T @ P @ A)
    
    return K, P

# Cost matrices
Q_pos = 10.0   # Position error weight
Q_theta = 5.0  # Orientation error weight
Q = np.diag([Q_pos, Q_pos, Q_theta])

R_v = 1.0      # Linear velocity effort weight
R_omega = 1.0  # Angular velocity effort weight
R = np.diag([R_v, R_omega])

# Compute LQR gain for point stabilization
K_lqr, P_lqr = LQR.compute_gain(A_0, B_0, Q, R)

print("\nLQR Configuration:")
print(f"Q = diag([{Q_pos}, {Q_pos}, {Q_theta}]) (position, position, orientation)")
print(f"R = diag([{R_v}, {R_omega}]) (linear vel, angular vel)")
print(f"\nLQR Gain K =\n{K_lqr}")
print(f"\nCost-to-go P =\n{P_lqr}")

### LQR Controller

In [ ]:
def lqr_control(state, goal, K, v_max, omega_max):
    """
    LQR controller with velocity saturation.
    
    Args:
        state: Current state [x, y, theta]
        goal: Goal state [x_g, y_g, theta_g]
        K: LQR gain matrix
        v_max: Max linear velocity
        omega_max: Max angular velocity
    
    Returns:
        Control input [v, omega]
    """
    # Compute error
    error = state - goal
    
    # Normalize angle error to [-pi, pi]
    error[2] = np.arctan2(np.sin(error[2]), np.cos(error[2]))
    
    # LQR control law: u = -K * e
    u = -K @ error
    
    # Saturate control
    v = np.clip(u[0], -v_max, v_max)
    omega = np.clip(u[1], -omega_max, omega_max)
    
    return np.array([v, omega])

# Test LQR controller
test_state = np.array([0.5, 0.3, 0.2])
test_goal = np.array([0.0, 0.0, 0.0])
test_control = lqr_control(test_state, test_goal, K_lqr, V_MAX, OMEGA_MAX)

print(f"\nController Test:")
print(f"  State: {test_state}")
print(f"  Goal: {test_goal}")
print(f"  Control: v={test_control[0]:.3f} m/s, omega={test_control[1]:.3f} rad/s")

### PID Controller for Comparison

In [ ]:
class PIDController:
    """Simple PID controller for TurtleBot point stabilization."""
    
    def __init__(self, kp_linear, kp_angular, ki_angular=0.0, kd_angular=0.0):
        self.kp_linear = kp_linear
        self.kp_angular = kp_angular
        self.ki_angular = ki_angular
        self.kd_angular = kd_angular
        
        self.integral = 0.0
        self.prev_error = 0.0
    
    def control(self, state, goal, dt, v_max, omega_max):
        """
        PID control for point stabilization.
        
        Strategy:
        1. Turn to face the goal
        2. Drive towards goal
        3. Final orientation adjustment
        """
        dx = goal[0] - state[0]
        dy = goal[1] - state[1]
        distance = np.sqrt(dx**2 + dy**2)
        
        # Desired heading to goal
        desired_theta = np.arctan2(dy, dx)
        
        # Heading error
        theta_error = desired_theta - state[2]
        theta_error = np.arctan2(np.sin(theta_error), np.cos(theta_error))
        
        # Final orientation error (when close to goal)
        final_theta_error = goal[2] - state[2]
        final_theta_error = np.arctan2(np.sin(final_theta_error), np.cos(final_theta_error))
        
        # PID terms for angular control
        self.integral += theta_error * dt
        derivative = (theta_error - self.prev_error) / dt if dt > 0 else 0.0
        self.prev_error = theta_error
        
        # Control logic
        if distance > 0.1:  # Far from goal - drive towards it
            v = self.kp_linear * distance
            omega = self.kp_angular * theta_error + \
                    self.ki_angular * self.integral + \
                    self.kd_angular * derivative
        else:  # Close to goal - adjust final orientation
            v = 0.0
            omega = self.kp_angular * final_theta_error
        
        # Saturate
        v = np.clip(v, -v_max, v_max)
        omega = np.clip(omega, -omega_max, omega_max)
        
        return np.array([v, omega])
    
    def reset(self):
        """Reset integrator."""
        self.integral = 0.0
        self.prev_error = 0.0

# Create PID controller
pid_controller = PIDController(
    kp_linear=0.5,
    kp_angular=2.0,
    ki_angular=0.1,
    kd_angular=0.05
)

print("\nPID Controller Created:")
print(f"  kp_linear: {pid_controller.kp_linear}")
print(f"  kp_angular: {pid_controller.kp_angular}")
print(f"  ki_angular: {pid_controller.ki_angular}")
print(f"  kd_angular: {pid_controller.kd_angular}")

### Unicycle Dynamics Simulation

In [ ]:
def simulate_unicycle(state, control, dt, noise_std=0.01):
    """
    Simulate unicycle dynamics.
    
    Args:
        state: Current state [x, y, theta]
        control: Control input [v, omega]
        dt: Time step
        noise_std: Process noise standard deviation
    
    Returns:
        Next state [x, y, theta]
    """
    x, y, theta = state
    v, omega = control
    
    # Nonlinear dynamics
    x_next = x + v * np.cos(theta) * dt
    y_next = y + v * np.sin(theta) * dt
    theta_next = theta + omega * dt
    
    # Add process noise
    noise = np.random.randn(3) * noise_std
    
    # Normalize angle
    theta_next = np.arctan2(np.sin(theta_next), np.cos(theta_next))
    
    return np.array([x_next, y_next, theta_next]) + noise

print("Unicycle dynamics simulator ready.")

## Simulation and Results

### Multi-Waypoint Navigation with LQR

In [ ]:
# Initialize
x_lqr = [np.array([0.0, 0.0, 0.0])]  # Start at origin
u_lqr = []

current_goal_idx = 0
goal_threshold = 0.05  # Switch to next goal when within 5cm

print("Running LQR simulation...")
for k in range(N_steps):
    state = x_lqr[-1]
    goal = goals[current_goal_idx]
    
    # Check if goal reached
    error = state - goal
    error[2] = np.arctan2(np.sin(error[2]), np.cos(error[2]))
    distance = np.linalg.norm(error[:2])
    
    if distance < goal_threshold and np.abs(error[2]) < 0.1:
        if current_goal_idx < len(goals) - 1:
            current_goal_idx += 1
            print(f"  Goal {current_goal_idx} reached at t={k*dt:.1f}s, switching to goal {current_goal_idx+1}")
    
    # Compute control
    control = lqr_control(state, goals[current_goal_idx], K_lqr, V_MAX, OMEGA_MAX)
    u_lqr.append(control)
    
    # Simulate
    next_state = simulate_unicycle(state, control, dt)
    x_lqr.append(next_state)

x_lqr = np.array(x_lqr[:-1])  # Remove last state (no control for it)
u_lqr = np.array(u_lqr)

print(f"LQR simulation complete. Final position: [{x_lqr[-1, 0]:.3f}, {x_lqr[-1, 1]:.3f}, {x_lqr[-1, 2]:.3f}]")

### Multi-Waypoint Navigation with PID

In [ ]:
# Initialize
x_pid = [np.array([0.0, 0.0, 0.0])]  # Start at origin
u_pid = []
pid_controller.reset()

current_goal_idx = 0

print("Running PID simulation...")
for k in range(N_steps):
    state = x_pid[-1]
    goal = goals[current_goal_idx]
    
    # Check if goal reached
    error = state - goal
    error[2] = np.arctan2(np.sin(error[2]), np.cos(error[2]))
    distance = np.linalg.norm(error[:2])
    
    if distance < goal_threshold and np.abs(error[2]) < 0.1:
        if current_goal_idx < len(goals) - 1:
            current_goal_idx += 1
            pid_controller.reset()  # Reset integrator for new goal
            print(f"  Goal {current_goal_idx} reached at t={k*dt:.1f}s, switching to goal {current_goal_idx+1}")
    
    # Compute control
    control = pid_controller.control(state, goals[current_goal_idx], dt, V_MAX, OMEGA_MAX)
    u_pid.append(control)
    
    # Simulate
    next_state = simulate_unicycle(state, control, dt)
    x_pid.append(next_state)

x_pid = np.array(x_pid[:-1])  # Remove last state
u_pid = np.array(u_pid)

print(f"PID simulation complete. Final position: [{x_pid[-1, 0]:.3f}, {x_pid[-1, 1]:.3f}, {x_pid[-1, 2]:.3f}]")

### Visualization: Trajectory Comparison

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# XY Trajectory
ax = axes[0]
ax.plot(x_lqr[:, 0], x_lqr[:, 1], 'b-', linewidth=2, label='LQR', alpha=0.8)
ax.plot(x_pid[:, 0], x_pid[:, 1], 'r-', linewidth=2, label='PID', alpha=0.8)

# Plot goals
for i, goal in enumerate(goals):
    ax.scatter(goal[0], goal[1], c='green', s=200, marker='*', 
              edgecolors='black', linewidth=2, zorder=5, label='Goals' if i == 0 else '')
    ax.annotate(f'G{i+1}', (goal[0], goal[1]), xytext=(5, 5), 
               textcoords='offset points', fontsize=12, fontweight='bold')

# Start point
ax.scatter(0, 0, c='blue', s=100, marker='o', label='Start', zorder=5)

# Draw orientation arrows at intervals
arrow_interval = 20
arrow_scale = 0.15
for i in range(0, len(x_lqr), arrow_interval):
    ax.arrow(x_lqr[i, 0], x_lqr[i, 1], 
            arrow_scale * np.cos(x_lqr[i, 2]), 
            arrow_scale * np.sin(x_lqr[i, 2]),
            head_width=0.05, head_length=0.05, fc='blue', ec='blue', alpha=0.5)

for i in range(0, len(x_pid), arrow_interval):
    ax.arrow(x_pid[i, 0], x_pid[i, 1], 
            arrow_scale * np.cos(x_pid[i, 2]), 
            arrow_scale * np.sin(x_pid[i, 2]),
            head_width=0.05, head_length=0.05, fc='red', ec='red', alpha=0.5)

ax.set_xlabel('X (m)', fontsize=12)
ax.set_ylabel('Y (m)', fontsize=12)
ax.set_title('Multi-Waypoint Navigation', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.axis('equal')

# Time series: Position error
ax = axes[1]
t_vec = np.arange(len(x_lqr)) * dt

# Compute position error to current goal
def compute_position_error(states, goals_list, threshold=0.05):
    errors = []
    goal_idx = 0
    for state in states:
        goal = goals_list[goal_idx]
        error = state - goal
        error[2] = np.arctan2(np.sin(error[2]), np.cos(error[2]))
        distance = np.linalg.norm(error[:2])
        errors.append(distance)
        
        # Check goal switch
        if distance < threshold and np.abs(error[2]) < 0.1 and goal_idx < len(goals_list) - 1:
            goal_idx += 1
    return np.array(errors)

error_lqr = compute_position_error(x_lqr, goals)
error_pid = compute_position_error(x_pid, goals)

ax.plot(t_vec, error_lqr, 'b-', linewidth=2, label='LQR', alpha=0.8)
ax.plot(t_vec, error_pid, 'r-', linewidth=2, label='PID', alpha=0.8)
ax.axhline(goal_threshold, color='gray', linestyle='--', alpha=0.5, label='Goal threshold')
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Position Error (m)', fontsize=12)
ax.set_title('Tracking Error Over Time', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Control Input Analysis

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

t_vec = np.arange(len(u_lqr)) * dt

# Linear velocity
ax = axes[0]
ax.plot(t_vec, u_lqr[:, 0], 'b-', linewidth=2, label='LQR', alpha=0.8)
ax.plot(t_vec, u_pid[:, 0], 'r-', linewidth=2, label='PID', alpha=0.8)
ax.axhline(V_MAX, color='gray', linestyle='--', alpha=0.5, label='Limit')
ax.axhline(-V_MAX, color='gray', linestyle='--', alpha=0.5)
ax.set_ylabel('Linear Velocity (m/s)', fontsize=12)
ax.set_title('Control Inputs Comparison', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Angular velocity
ax = axes[1]
ax.plot(t_vec, u_lqr[:, 1], 'b-', linewidth=2, label='LQR', alpha=0.8)
ax.plot(t_vec, u_pid[:, 1], 'r-', linewidth=2, label='PID', alpha=0.8)
ax.axhline(OMEGA_MAX, color='gray', linestyle='--', alpha=0.5, label='Limit')
ax.axhline(-OMEGA_MAX, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Angular Velocity (rad/s)', fontsize=12)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistics
print("\n=== Control Effort Statistics ===")
print("\nLQR:")
print(f"  Linear velocity: mean={np.mean(np.abs(u_lqr[:, 0])):.3f}, max={np.max(np.abs(u_lqr[:, 0])):.3f} m/s")
print(f"  Angular velocity: mean={np.mean(np.abs(u_lqr[:, 1])):.3f}, max={np.max(np.abs(u_lqr[:, 1])):.3f} rad/s")
print(f"  Total control effort: {np.sum(u_lqr[:, 0]**2 + u_lqr[:, 1]**2):.2f}")

print("\nPID:")
print(f"  Linear velocity: mean={np.mean(np.abs(u_pid[:, 0])):.3f}, max={np.max(np.abs(u_pid[:, 0])):.3f} m/s")
print(f"  Angular velocity: mean={np.mean(np.abs(u_pid[:, 1])):.3f}, max={np.max(np.abs(u_pid[:, 1])):.3f} rad/s")
print(f"  Total control effort: {np.sum(u_pid[:, 0]**2 + u_pid[:, 1]**2):.2f}")

### Performance Metrics

In [ ]:
# Compute cumulative cost
def compute_cumulative_cost(states, controls, goals_list, Q, R, threshold=0.05):
    """
    Compute cumulative LQR cost: sum(e'*Q*e + u'*R*u)
    """
    cost = 0.0
    goal_idx = 0
    
    for i in range(len(controls)):
        state = states[i]
        control = controls[i]
        goal = goals_list[goal_idx]
        
        # Error
        error = state - goal
        error[2] = np.arctan2(np.sin(error[2]), np.cos(error[2]))
        
        # Cost
        cost += error @ Q @ error + control @ R @ control
        
        # Check goal switch
        distance = np.linalg.norm(error[:2])
        if distance < threshold and np.abs(error[2]) < 0.1 and goal_idx < len(goals_list) - 1:
            goal_idx += 1
    
    return cost

cost_lqr = compute_cumulative_cost(x_lqr, u_lqr, goals, Q, R)
cost_pid = compute_cumulative_cost(x_pid, u_pid, goals, Q, R)

print("\n=== Performance Comparison ===")
print(f"\nCumulative LQR Cost:")
print(f"  LQR: {cost_lqr:.2f}")
print(f"  PID: {cost_pid:.2f}")
print(f"  LQR improvement: {(cost_pid - cost_lqr) / cost_pid * 100:.1f}% lower cost")

# Tracking performance
print(f"\nPosition Tracking Error:")
print(f"  LQR: mean={np.mean(error_lqr):.4f} m, max={np.max(error_lqr):.4f} m")
print(f"  PID: mean={np.mean(error_pid):.4f} m, max={np.max(error_pid):.4f} m")

# Settling time (time to reach within threshold of final goal)
final_goal = goals[-1]
settling_idx_lqr = np.where(error_lqr < goal_threshold)[0]
settling_idx_pid = np.where(error_pid < goal_threshold)[0]

if len(settling_idx_lqr) > 0:
    settling_time_lqr = settling_idx_lqr[-1] * dt
    print(f"\nSettling Time (to final goal):")
    print(f"  LQR: {settling_time_lqr:.1f} s")

if len(settling_idx_pid) > 0:
    settling_time_pid = settling_idx_pid[-1] * dt
    print(f"  PID: {settling_time_pid:.1f} s")

## Integration with pykal's DynamicalSystem

In [ ]:
from pykal import DynamicalSystem
from pykal.algorithm_library.controllers.lqr import LQR

# Create LQR controller as DynamicalSystem
lqr_system = DynamicalSystem(
    f=LQR.f,
    state_name='state',
    h=lambda u, **kwargs: u  # Control output
)

# Unicycle plant
def unicycle_dynamics(state, u, dt, **kwargs):
    """Unicycle dynamics."""
    x, y, theta = state
    v, omega = u
    
    x_next = x + v * np.cos(theta) * dt
    y_next = y + v * np.sin(theta) * dt
    theta_next = theta + omega * dt
    theta_next = np.arctan2(np.sin(theta_next), np.cos(theta_next))
    
    return np.array([x_next, y_next, theta_next])

unicycle_plant = DynamicalSystem(
    f=unicycle_dynamics,
    state_name='state',
    h=lambda state, **kwargs: state
)

# Simulation with DynamicalSystem composition
param_dict = {
    'state': np.array([0.0, 0.0, 0.0]),
    'goal': np.array([1.0, 1.0, np.pi/4]),
    'K': K_lqr,
    'dt': dt
}

print("\n=== DynamicalSystem Integration Test ===")
for k in range(5):
    # Compute control
    error = param_dict['state'] - param_dict['goal']
    error[2] = np.arctan2(np.sin(error[2]), np.cos(error[2]))
    u = -param_dict['K'] @ error
    u = np.clip(u, [-V_MAX, -OMEGA_MAX], [V_MAX, OMEGA_MAX])
    
    param_dict['u'] = u
    
    # Update plant
    next_state = unicycle_plant.step(params=param_dict)
    
    print(f"Step {k}: state=[{next_state[0]:.3f}, {next_state[1]:.3f}, {next_state[2]:.3f}], "
          f"control=[{u[0]:.3f}, {u[1]:.3f}]")

print("\n✓ LQR successfully integrated with DynamicalSystem framework")

## ROS2 Deployment

### Creating LQR Node for TurtleBot3

In [ ]:
from pykal import ROSNode
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Twist, PoseStamped

# LQR callback for ROS2
def lqr_callback(tk, odom, goal, **kwargs):
    """
    LQR control callback for ROS2.
    
    Args:
        tk: Current time (s)
        odom: Odometry message [x, y, theta] from /odom topic
        goal: Goal pose [x, y, theta] from /goal topic
    
    Returns:
        dict with 'cmd_vel' key -> Twist message
    """
    # Extract state
    state = np.array([odom[0], odom[1], odom[2]])  # [x, y, theta]
    goal_state = np.array([goal[0], goal[1], goal[2]])
    
    # Compute error
    error = state - goal_state
    error[2] = np.arctan2(np.sin(error[2]), np.cos(error[2]))
    
    # LQR control
    u = -kwargs['K'] @ error
    
    # Saturate
    v = np.clip(u[0], -kwargs['v_max'], kwargs['v_max'])
    omega = np.clip(u[1], -kwargs['omega_max'], kwargs['omega_max'])
    
    # Return as velocity command array [linear.x, linear.y, linear.z, angular.x, angular.y, angular.z]
    cmd_vel = np.array([v, 0.0, 0.0, 0.0, 0.0, omega])
    
    return {'cmd_vel': cmd_vel}

# Create ROS node (example - not executed in notebook)
lqr_node_config = {
    'callback': lqr_callback,
    'subscriptions': [
        ('/odom', Odometry, 'odom'),
        ('/goal_pose', PoseStamped, 'goal')
    ],
    'publications': [
        ('cmd_vel', Twist, '/cmd_vel')
    ],
    'param_dict': {
        'K': K_lqr,
        'v_max': V_MAX,
        'omega_max': OMEGA_MAX
    },
    'node_name': 'turtlebot3_lqr_controller',
    'rate': 10  # 10 Hz control rate
}

print("\n=== ROS2 Node Configuration ===")
print(f"Node name: {lqr_node_config['node_name']}")
print(f"Control rate: {lqr_node_config['rate']} Hz")
print(f"Subscriptions: {len(lqr_node_config['subscriptions'])} topics")
print(f"Publications: {len(lqr_node_config['publications'])} topics")
print("\nTo deploy:")
print("  1. lqr_node = ROSNode(**lqr_node_config)")
print("  2. lqr_node.create_node()")
print("  3. lqr_node.start()")
print("  4. # ... robot navigates to goals ...")
print("  5. lqr_node.stop()")

## Conclusion

### Key Results

This notebook demonstrated LQR control for TurtleBot3 point stabilization with the following findings:

1. **Optimality**: LQR achieves lower cumulative cost compared to PID by balancing tracking error and control effort optimally

2. **Smooth Control**: LQR generates smoother control signals due to the cost function penalizing control effort, reducing actuator wear

3. **Guaranteed Stability**: For linear systems, LQR guarantees closed-loop stability with the computed feedback gain

4. **Systematic Tuning**: The Q and R matrices provide intuitive tuning parameters (state vs control weighting) compared to PID's three independent gains per axis

5. **Performance Trade-off**: LQR typically achieves better performance than PID in terms of the LQR cost function, though PID may converge faster in some cases

### When to Use LQR

**Use LQR when:**
- System dynamics can be reasonably linearized
- You want to optimize a specific cost function (tracking + effort)
- Smooth control signals are important (battery life, actuator wear)
- Guaranteed stability is required
- Systematic tuning is preferred over trial-and-error

**Use PID when:**
- System model is unknown or highly uncertain
- Simple implementation is critical
- Aggressive response is more important than optimality
- Working with legacy systems already tuned for PID

### Practical Applications

- **Mobile Robot Navigation**: Warehouse robots, delivery robots
- **Trajectory Tracking**: Following paths in structured environments
- **Formation Control**: Multi-robot coordination
- **Docking**: Precision positioning for charging stations

### Next Steps

1. **Time-Varying LQR**: Handle trajectory tracking (not just point stabilization)
2. **Adaptive LQR**: Update gains online based on system identification
3. **Robust LQR**: Add robustness to model uncertainties
4. **Hardware Testing**: Deploy on physical TurtleBot3 with Gazebo validation
5. **Obstacle Avoidance**: Combine with path planning algorithms

### References

For the theoretical foundation and implementation details, see:
- LQR theory: :cite:`anderson2007optimal`
- Mobile robot control: :cite:`siegwart2011introduction`
- TurtleBot3 documentation